<a href="https://colab.research.google.com/github/TakehikoEsaka/kaggle_digit_recognizer/blob/master/cnn_library_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初心者のための TensorFlow 2.0 入門

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

この短いイントロダクションでは [Keras](https://www.tensorflow.org/guide/keras/overview) を使って下記のことを行います。

1. 画像を分類するニューラルネットワークを構築する
2. このニューラルネットワークを訓練する
3. そして最後に、モデルの正解率を評価する

このファイルは [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) の notebook ファイルです。 Python プログラムはブラウザ上で直接実行されます。TensorFlow を学んだり使ったりするには最良の方法です。Google Colab のnotebook の実行方法は以下のとおりです。

1. Pythonランタイムへの接続：メニューバーの右上で「接続」を選択します。
2. ノートブックのコードセルをすべて実行：「ランタイム」メニューから「すべてのセルを実行」を選択します。

TensorFlow 2 のパッケージをダウンロードしてインストールします。プログラムに TensorFlow をインポートします。

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow をインストール
try:
  # %tensorflow_version は Colab 上でのみ利用可能
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

[MNIST データセット](http://yann.lecun.com/exdb/mnist/)をロードして準備します。サンプルを整数から浮動小数点数に変換します。

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

層を積み重ねて`tf.keras.Sequential`モデルを構築します。訓練のためにオプティマイザと損失関数を選びます。

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [1]:
#Attention Branch Network

# 分岐型ネットワークでの定義
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,\
                        MaxPool2D, GlobalAveragePooling2D, BatchNormalization

from tensorflow.keras import backend as K

original_im = Input(shape=(28,28,1))
x = Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same',
           activation='relu', input_shape = (28,28,1))(original_im)
x = Conv2D(filters = 64, kernel_size = (5,5,), padding = "Same",
           activation="relu")(x)
x1 = MaxPool2D(pool_size = (2,2), strides = (2,2))(x) #14*14*64

# Attention Branch
x2 = Conv2D(filters = 64, kernel_size = (5,5,), padding = "Same",
           activation="relu")(x1)
x2 = Conv2D(filters = 64, kernel_size = (1,1,), padding = "Same",
           activation="relu")(x2)

# Attention Branch 1
x2_1 = Conv2D(filters = 64, kernel_size = (1,1,), padding = "Same",
           activation="sigmoid")(x2) #14*14*64

# Attention Branch 2
x2_2 = Conv2D(filters = 64, kernel_size = (1,1,), padding = "Same",
           activation="relu")(x2)
x2_2 = GlobalAveragePooling2D(data_format = None)(x2_2) 
output = Dense(10, activation = "softmax")(x2_2)

# Perception Branch
#ここにx1とx2_1を足し合わせる記述を記載
x4 = tf.keras.layers.multiply([x1, x2_1]) #アダマール積
x4 = tf.keras.layers.add([x1, x4 + 1])
x4 = Dense(512, activation = "relu")(x4) 
output = Dense(10, activation = "softmax")(x4)

model = tf.keras.models.Model(inputs = original_im, outputs = output)

NameError: name 'tf' is not defined

In [ ]:
mo

In [0]:
# FCN
# Todo : 
# ★model.summary()で画像サイズを見る。
# ★画像の切り出しについて調べる。（labelmeをインストールhttps://github.com/wkentaro/labelme）
# ★実際に学習と予測をしてみる。

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, AveragePooling2D, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy

import tensorflow as tf
from keras.engine.topology import Layer
from keras.engine import InputSpec

class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        n = s[0]
        if s[1] == None:
            h = None
        else:
            h = s[1] + 2 * self.padding[0]
        if s[2] == None:
            w = None
        else:
            w = s[2] + 2 * self.padding[1]
        c = s[3]
        return (n, h, w, c)

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

class UserModel(object):

    def __init__(self, cut_size, channel, category_count, optimizer):
        self.cut_size = cut_size
        self.channel = channel
        self.category_count = category_count
        self.optimizer = optimizer

    def __denseBlock(self, inputT, channels = 32):

        out = ReflectionPadding2D((1,1))(inputT)
        out = Conv2D(channels, (3, 3), activation='relu', padding='valid')(out)
        out = Concatenate()([out, inputT])
        out = Conv2D(channels, (1, 1), activation='relu')(out)
        out = BatchNormalization()(out)
        return out

    def __modelBody(self, inputT):

        # 56*56
        x = BatchNormalization()(inputT)
        x = ReflectionPadding2D((2,2))(x)
        x = Conv2D(16, (5, 5), strides=2, activation='relu', padding='valid')(x)
        x = BatchNormalization()(x)

        # 28*28
        x = self.__denseBlock(x)
        x = self.__denseBlock(x)
        x = MaxPooling2D((2, 2))(x)


        # 14*14
        x = self.__denseBlock(x)
        x = self.__denseBlock(x)
        x = MaxPooling2D((2, 2))(x)

        # 7*7
        x = self.__denseBlock(x)
        x = self.__denseBlock(x)
        x = self.__denseBlock(x)

        return x

    def __clsModelSetup(self, input_img):

        x = self.__modelBody(input_img)

        x = AveragePooling2D((7, 7))(x)
        x = Conv2D(self.category_count, (1, 1), activation='softmax', padding='valid')(x)

        out = Flatten()(x)

        return out

    def __fcnModelSetup(self, input_img):

        x = self.__modelBody(input_img)

        x = AveragePooling2D((7, 7), strides=1, padding='same')(x)

        out = Conv2D(self.category_count, (1, 1), activation='softmax', padding='valid')(x)

        return out

    def getTrainModel(self):

        input_img = Input(shape=(self.cut_size['height'], self.cut_size['width'], self.channel))
        result = self.__clsModelSetup(input_img)
        model = Model(input_img, result)
        model.compile(optimizer=self.optimizer, loss='binary_crossentropy', metrics=['categorical_accuracy'])

        return model

    def getCLSModel(self):

        input_img = Input(shape=(self.cut_size['height'], self.cut_size['width'], self.channel))
        result = self.__clsModelSetup(input_img)
        model = Model(input_img, result)

        return model

    def getFCNModel(self):

        input_img = Input(shape=(None, None, self.channel))
        result = self.__fcnModelSetup(input_img)
        model = Model(input_img, result)

        return model



In [0]:
# Autoencoder.変分下限いれたものがVAEになる。

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

input_img = Input(shape=(28, 28, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
# GANの基本的な構造
class GAN():
    def __init__(self):
        #mnistデータ用の入力データサイズ
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # 潜在変数の次元数 
        self.z_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # discriminatorモデル
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Generatorモデル
        self.generator = self.build_generator()
        # generatorは単体で学習しないのでコンパイルは必要ない
        #self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        self.combined = self.build_combined1()
        #self.combined = self.build_combined2()
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (self.z_dim,)
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        return model

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def build_combined1(self):
        self.discriminator.trainable = False
        model = Sequential([self.generator, self.discriminator])
        return model

    def build_combined2(self):
        z = Input(shape=(self.z_dim,))
        img = self.generator(z)
        self.discriminator.trainable = False
        valid = self.discriminator(img)
        model = Model(z, valid)
        model.summary()
        return model

In [0]:
def train(self, epochs, batch_size=128, save_interval=50):

        # mnistデータの読み込み
        (X_train, _), (_, _) = mnist.load_data()

        # 値を-1 to 1に規格化
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Discriminatorの学習
            # ---------------------

            # バッチサイズの半数をGeneratorから生成
            noise = np.random.normal(0, 1, (half_batch, self.z_dim))
            gen_imgs = self.generator.predict(noise)


            # バッチサイズの半数を教師データからピックアップ
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # discriminatorを学習
            # 本物データと偽物データは別々に学習させる
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            # それぞれの損失関数を平均
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Generatorの学習
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.z_dim))

            # 生成データの正解ラベルは本物（1） 
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # 進捗の表示
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # 指定した間隔で生成画像を保存
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

モデルを訓練してから評価します。

In [0]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

この画像分類器は、今回のデータセットで訓練した場合、最大98%程度の正解率となります。更に学ぶには[TensorFlow tutorials](https://www.tensorflow.org/tutorials/)を読んでください。